This Code is for solo song info

In [12]:
import requests
import lyricsgenius

access_token = 'j81yhRG0w5EWG59FUWPNHAPuUwuTkEPf6R-GySZ92XSyb4zWUziTZ-6uJLFrIfvI'
genius = lyricsgenius.Genius(access_token, timeout=15)
base_url = 'https://api.genius.com'

def get_song_id(song_title, artist_name):
    search_url = f'{base_url}/search'
    headers = {'Authorization': f'Bearer {access_token}'}
    params = {'q': f'{song_title} {artist_name}'}

    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code == 200:
        search_results = response.json()

        if search_results['response']['hits']:
            song_path = search_results['response']['hits'][0]['result']['path']
            song_id = search_results['response']['hits'][0]['result']['id']
            song_url = f'https://genius.com{song_path}'
            
            return song_id, song_url
        else:
            return None, None
    else:
        return None, None

artist_name = "Daddy Yankee"
song_title = "REMIX"
song_id, song_url = get_song_id(song_title, artist_name)

if song_id:
    print(f"Song ID: {song_id}")
    lyrics = genius.lyrics(song_id)
    print("\nSongtext:\n")
    print(lyrics)
else:
    print("Song nicht gefunden.")

Song ID: 3057010

Songtext:

[Letra de "Despacito (Remix)" ft. Justin Bieber]

[Intro: Justin Bieber]
Come and move that in my direction
So thankful for that, it's such a blessin', yeah
Turn every situation into heaven, yeah
Oh-oh, you are
My sunrise on the darkest day
Got me feelin' some kind of way
Make me wanna savor every moment slowly, slowly
You fit me tailor-made, love how you put it on
Got the only key, know how to turn it on
The way you nibble on my ear, the only words I wanna hear
Baby, take it slow so we can last long

[Verso 1: Luis Fonsi & Daddy Yankee]
¡Oh! Tú, tú eres el imán y yo soy el metal
Me voy acercando y voy armando el plan
Solo con pensarlo, se acelera el pulso (Oh, yeah)
Ya, ya me está gustando más de lo normal
Todos mis sentidos van pidiendo más
Esto hay que tomarlo sin ningún apuro

[Coro: Justin Bieber & Luis Fonsi, Daddy Yankee]
Despacito
Quiero respirar tu cuello despacito
Deja que te diga cosas al oído
Para que te acuerdes si no estás conmigo
Despacito
Qu

This Code is for lyric searching for all songs

In [ ]:
import pandas as pd
import requests
import lyricsgenius

# authentification
access_token = 'j81yhRG0w5EWG59FUWPNHAPuUwuTkEPf6R-GySZ92XSyb4zWUziTZ-6uJLFrIfvI'
genius = lyricsgenius.Genius(access_token, timeout=30)
base_url = 'https://api.genius.com'

# get song_id for lyric search
def get_song_id(song_title, artist_name):
    search_url = f'{base_url}/search'
    headers = {'Authorization': f'Bearer {access_token}'}
    params = {'q': f'{song_title} {artist_name}'}

    try:
        response = requests.get(search_url, headers=headers, params=params, timeout=30, stream=True)
        if response.status_code == 200:
            search_results = response.json()

            if search_results['response']['hits']:
                song_path = search_results['response']['hits'][0]['result']['path']
                song_id = search_results['response']['hits'][0]['result']['id']
                song_url = f'https://genius.com{song_path}'

                return song_id, song_url
            else:
                return None, None
        else:
            return None, None
    except requests.exceptions.Timeout:
        print(f"Timeout for {song_title} from {artist_name}.")
        return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None, None

# read in songs for adding lyrics
df = pd.read_csv('/Users/juliusriss/Desktop/data-science-project-local/data/cleaned_global_17-24_top_ten_songs.csv')
df['temp_artist_names'] = df['artist_names'].apply(lambda x: x.split(',')[0] if ',' in x else x)
df['temp_track_name'] = df['track_name'].apply(lambda x: x.split('(')[0].split('-')[0].strip())

lyrics_list = []

# adding lyrics
for index, row in df.iterrows():
    artist_name = row['temp_artist_names']
    song_title = row['temp_track_name']

    print(f'Searching Song: {song_title} von {artist_name}')

    song_id, song_url = get_song_id(song_title, artist_name)
    
    if song_id:
        try:
            lyrics = genius.lyrics(song_id)
            lyrics_list.append(lyrics)
            print("-> Added Lyrics\n")
        except Exception as e:
            print(f"Error adding lyrics {song_title}: {e}")
            lyrics_list.append(None)
    else:
        lyrics_list.append(None)
        print("-> Didnt find song\n")

# saving new file with added lyrics
df['lyrics'] = lyrics_list
df = df.drop(columns=['temp_artist_names', 'temp_track_name'])
df.to_csv('/Users/juliusriss/Desktop/data-science-project-local/data/cleaned_global_17-24_top_ten_songs_with_lyrics.csv', index=False)

Searching Song: Blinding Lights von The Weeknd
-> Added Lyrics

Searching Song: Shape of You von Ed Sheeran
-> Added Lyrics

Searching Song: STAY von The Kid LAROI


KeyboardInterrupt: 